In [ ]:
import scipy as sp
from scipy.integrate import odeint
from scipy.integrate import quad
from scipy.integrate import cumulative_trapezoid
from scipy.integrate import solve_ivp
from scipy.misc import derivative
from scipy.interpolate import interp1d
from scipy.optimize import fsolve, root_scalar

import sympy as smp
from sympy import symbols, Eq, solve, exp, sqrt, latex, log, integrate, Function, nsolve, srepr, parse_expr, Piecewise, plot, preorder_traversal, collect
import math

import numpy as np
import pandas as pd

import pickle
import matplotlib.pyplot as plt
from IPython.display import display

from datetime import datetime

In [1]:
 def findLogArgs(expr):
        if expr.func == smp.log:
            return [expr.args[0]]  
        log_args = []
        for arg in expr.args:
            log_args.extend(findLogArgs(arg))  
        return log_args

def ConvertIndefinate2Definate(expr,limits):    
    def Replacer(expr,limits):
        dx=expr.variables
        integrand=expr.function
        result=smp.Integral(integrand,(dx,limits[0],limits[1]))
        return result
   
    for arg in expr.args:            
        if isinstance(arg, smp.Integral):
            replacer=Replacer(arg,limits)
            expr=expr.subs(arg,replacer) 

    logArguments=findLogArgs(expr)
    for logs in logArguments: 
        for arg in logs.args:
            if isinstance(arg, smp.Integral):
                replacer=Replacer(arg,limits)
                expr=expr.subs(arg,replacer) 
                
    return expr


def ConvertIndefinate2DefinateDeep(Expr,Limits): 
    def HasNestedIntegral(expr):
        for node in preorder_traversal(expr):
            if isinstance(node, smp.Integral):
                return True
        return False
        
    def ConvertIndefinate2Definate1(expr,limits):
        def Replacer(expr,limits):
            dx=expr.variables
            integrand=expr.function
            result=smp.Integral(integrand,(dx,limits[0],limits[1]))
            return result
            
        replaced=False
        for node in preorder_traversal(expr):
            if isinstance(node, smp.Integral) and len(node.args[1])==1:
                isnested=HasNestedIntegral(node.function)
                if(True):
                    replacer=Replacer(node,limits)
                    expr=expr.replace(node,replacer)
                    # display(node,'-----',replacer)
                    replaced=True
                    break
        return expr,replaced
    r=True
    while(r):
        Expr,r=ConvertIndefinate2Definate1(Expr,Limits)
    return Expr

In [2]:
def IntegralDeepClean(Expr): 
    def HasNestedIntegral(expr):
        for node in preorder_traversal(expr):
            if isinstance(node, smp.Integral):
                return True
        return False
    
    for node in preorder_traversal(Expr):
         if isinstance(node, smp.Integral):
             nested=HasNestedIntegral(node.function)
             if(not nested):
                 simple=node.simplify()
                 Expr=Expr.replace(node,simple)
    return Expr

def IntegrandDeepClean(Expr): 
    def HasNestedIntegral(expr):
        for node in preorder_traversal(expr):
            if isinstance(node, smp.Integral):
                return True
        return False
    
    for node in preorder_traversal(Expr):
         if isinstance(node, smp.Integral):
             nested=HasNestedIntegral(node.function)
             if(not nested):
                 simple=node.function.simplify()
                 Expr=Expr.replace(node.function,simple)
    return Expr


def IntegralDeepSepration(Expr):
    def dealExponent(expr,dx):
        fun=expr.args[0]
        seps=fun.as_independent(dx,as_Mul=True)
        seprated=exp(seps[0])*exp(seps[1])
        return seprated
        
    def HasNestedIntegral(expr):
        for node in preorder_traversal(expr):
            if isinstance(node, smp.Integral):
                return True
        return False

    def UpgradeIntegral(expr,integrand):
        dx=expr.limits[0]
        return smp.Integral(integrand,(dx[0],dx[1],dx[2])).simplify()
    
    for node in preorder_traversal(Expr):
         if isinstance(node, smp.Integral):
             isnested=HasNestedIntegral(node.function)
             if(not isnested):
                 fun=node.function
                 dx=node.variables[0]
                 for n in preorder_traversal(fun):
                     if n.func == exp:
                         exed= dealExponent(n,dx)
                         fun=fun.replace(n,exed)
                 updatedNode=UpgradeIntegral(node,fun)
                 Expr=Expr.replace(node,updatedNode)
                
    return Expr

def LogCleaning(Expr):
    for node in preorder_traversal(Expr):
        if node.func==log:
            if(len(node.args)==1 and node.args[0].func==exp):        
                Expr=Expr.subs(node,node.args[0].args[0])
            else:
                expanded=smp.expand_log(node,force=True)
                Expr=Expr.subs(node,expanded)
        
    return Expr
